<a href="https://colab.research.google.com/github/Suraez/laughing-octo-sniffle/blob/main/expt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is for converting the downloaded raw .txt data to .csv data, Ignore this if you have already got the CSV

In [ ]:
import pandas as pd

# Specify the input TXT file path and output CSV file path
txt_file_path = "/content/data_2021_azure.txt"
csv_file_path = "/content/data.csv"

# Load the TXT file into a pandas DataFrame
# Assuming the TXT file is comma-separated
df = pd.read_csv(txt_file_path, sep=",")

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"File converted and saved as {csv_file_path}")


File converted and saved as /content/data.csv


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('/content/drive/MyDrive/research/data.csv')

# Sort by app, func, and end_timestamp to ensure order
df = df.sort_values(by=['app', 'func', 'end_timestamp'])

# Calculate time interval in seconds between invocations for each function
df['time_interval'] = df.groupby(['app', 'func'])['end_timestamp'].diff()
df = df.dropna(subset=['time_interval'])  # Remove NaNs resulting from the first invocation


In [ ]:
intervals = df['time_interval'].values


In [ ]:
sequence_length = 10  # Adjust as needed
X = np.lib.stride_tricks.sliding_window_view(intervals, sequence_length)
y = intervals[sequence_length:]

In [ ]:
# Reshape X for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshaping to (samples, sequence_length, features)
# Ensure X and y have consistent lengths
X = X[:-1]


### Normalization of X and Y

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)
y = scaler.fit_transform(y.reshape(-1, 1))


In [ ]:
# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),  # Regularization to prevent overfitting
    Dense(50, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2,
                    callbacks=[early_stopping], verbose=1)


Epoch 1/2
39611/39611 ━━━━━━━━━━━━━━━━━━━━ 258s 7ms/step - loss: 3.0607e-05 - val_loss: 2.2008e-05
Epoch 2/2
39611/39611 ━━━━━━━━━━━━━━━━━━━━ 256s 6ms/step - loss: 3.0266e-05 - val_loss: 2.1416e-05


In [ ]:
# Example usage
app_id = '85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031'  # Replace with the actual app ID
func_id = 'e02465de583b6ceffa5b78cce5f10eb27e714a8a6b3aed483be50f30a924071f'  # Replace with the actual function ID
next_invocation, predicted_interval = predict_next_invocation(model, X, df, app_id, func_id)
print(f"Predicted next invocation timestamp: {next_invocation}")
print(f"Predicted time interval: {predicted_interval}")

NameError: name 'predict_next_invocation' is not defined

In [ ]:
!git clone https://github.com/Suraez/laughing-octo-sniffle.git
%cd laughing-octo-sniffle
!git status


Cloning into 'laughing-octo-sniffle'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 4.92 KiB | 4.92 MiB/s, done.
/content/laughing-octo-sniffle
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


### Code for counting the unique value in a column

In [ ]:
# Replace 'file_path' with the path to your CSV file
file_path = '/content/sampled_dataset.csv'
# Replace 'column_name' with the name of the column you want to analyze
column_name = 'HashFunction'

def count_unique_values(file_path, column_name):
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Count the number of unique values in the specified column
        unique_count = df[column_name].nunique()
        return unique_count
    except Exception as e:
        print(f"Error: {e}")
        return None

unique_count = count_unique_values(file_path, column_name)

if unique_count is not None:
    print(f"Number of unique values in column '{column_name}': {unique_count}")


Number of unique values in column 'HashFunction': 40


### Sampling the Azure Function Invocation Traceset 2019

In [ ]:
# Assume 'file_path' is the path to your dataset
file_path = '/content/drive/MyDrive/research/data2019.csv'

# Load the dataset
df = pd.read_csv(file_path)

# Step 1: Select the first 480 minute columns (representing the 8-hour period)
# The minute columns are assumed to start at index 3 (after 'HashOwner', 'HashApp', 'HashFunction')
start_minute = 4  # Starting index for minute columns
end_minute = start_minute + 480  # Select 480 columns for 8 hours
selected_columns = ['HashOwner','HashApp', 'HashFunction', 'Trigger'] + df.columns[start_minute:end_minute].tolist()

df_subset = df[selected_columns]

# Step 2: Randomly sample 20 unique functions
# Get unique functions from the 'HashFunction' column
unique_functions = df_subset['HashFunction'].unique()

# Randomly select 20 functions
sampled_functions = pd.Series(unique_functions).sample(n=20, random_state=42).tolist()

# Step 3: Filter the dataset for the sampled functions
filtered_df = df_subset[df_subset['HashFunction'].isin(sampled_functions)]

# Save the sampled dataset to a new file (optional)
filtered_df.to_csv('sampled_dataset.csv', index=False)

print("Dataset sampled for 8 hours and 20 functions. Saved to 'sampled_dataset.csv'.")


Dataset sampled for 8 hours and 20 functions. Saved to 'sampled_dataset.csv'.


### printing out rows having values greater than 1

In [ ]:

# Step 1: Select the relevant columns (index 4 to 483)
numeric_columns = filtered_df.columns[3:483]  # Column indices are 0-based

# Step 2: Filter rows where any value in the specified columns is greater than 1
rows_with_values_gt_1 = filtered_df[(filtered_df[numeric_columns] > 1).any(axis=1)]

# Step 3: Print the filtered rows
print(rows_with_values_gt_1)

# Save to a CSV file (optional)
rows_with_values_gt_1.to_csv('rows_with_values_gt_1.csv', index=False)
